In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Attention, Embedding, Flatten, LayerNormalization, MultiHeadAttention


In [2]:
# Load the data
aapl_df = pd.read_pickle('/Raw Data/pickle/aapl_prices_labels_news.pkl')
amzn_df = pd.read_pickle('/Raw Data/pickle/amzn_prices_labels_news.pkl')
msft_df = pd.read_pickle('/Raw Data/pickle/msft_prices_labels_news.pkl')

In [3]:
# Choose the features and target
def data_split(df):
  features = df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'neg', 'neu', 'pos', 'compound', 'subjectivity', 'polarity']]
  target = df['Adj Close Next']

  # Split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, shuffle=False)

  # Standardize the features
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  # Reshape the data for the transformer model
  X_train = np.expand_dims(X_train, axis=1)
  X_test = np.expand_dims(X_test, axis=1)

  return X_train, X_test, y_train, y_test



In [4]:
# Custom Attention Layer
class MyAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(MyAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.Wq = self.add_weight(name="Wq", shape=(input_shape[-1], input_shape[-1]), initializer="uniform", trainable=True)
        self.Wk = self.add_weight(name="Wk", shape=(input_shape[-1], input_shape[-1]), initializer="uniform", trainable=True)
        self.Wv = self.add_weight(name="Wv", shape=(input_shape[-1], input_shape[-1]), initializer="uniform", trainable=True)

    def call(self, inputs):
        q = tf.matmul(inputs, self.Wq)
        k = tf.matmul(inputs, self.Wk)
        v = tf.matmul(inputs, self.Wv)

        attention_weights = tf.nn.softmax(tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(tf.cast(tf.shape(k)[-1], tf.float32)))
        output = tf.matmul(attention_weights, v)

        return output

In [5]:
aapl_X_train, aapl_X_test, aapl_y_train, aapl_y_test = data_split(aapl_df)
amzn_X_train, amzn_X_test, amzn_y_train, amzn_y_test = data_split(amzn_df)
msft_X_train, msft_X_test, msft_y_train, msft_y_test = data_split(msft_df)

In [6]:
# Define the Transformer model using the custom attention layer
def create_transformer_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        MyAttention(),
        Dropout(0.2),
        LayerNormalization(epsilon=1e-6),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(1)  # Output layer
    ])
    return model


In [7]:
# Create the model
aapl_model = create_transformer_model(aapl_X_train.shape[1:])
amzn_model = create_transformer_model(amzn_X_train.shape[1:])
msft_model = create_transformer_model(msft_X_train.shape[1:])

In [8]:
# Compile the model
models = {'AAPL':[aapl_model, aapl_X_train, aapl_X_test, aapl_y_train, aapl_y_test],
          'AMZN': [amzn_model, amzn_X_train, amzn_X_test, amzn_y_train, amzn_y_test],
          'MSFT': [msft_model, msft_X_train, msft_X_test, msft_y_train, msft_y_test]}

for model in models:
  print(f"{model}")

  transformer = models[model][0]
  X_train = models[model][1]
  X_test = models[model][2]
  y_train = models[model][3]
  y_test = models[model][4]

  transformer.compile(optimizer='adam', loss='mean_squared_error', metrics = ['mae', 'mape'])

  # Train the model
  transformer.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)
  print("\n\n\n\n")


AAPL
Epoch 1/50
6/6 [==============================] - 7s 117ms/step - loss: 24419.4766 - mae: 155.3483 - mape: 99.7869 - val_loss: 27562.1074 - val_mae: 165.9826 - val_mape: 99.6389
Epoch 2/50
6/6 [==============================] - 0s 20ms/step - loss: 24191.6562 - mae: 154.6019 - mape: 99.2844 - val_loss: 27345.3184 - val_mae: 165.3286 - val_mape: 99.2463
Epoch 3/50
6/6 [==============================] - 0s 20ms/step - loss: 23931.0371 - mae: 153.7536 - mape: 98.7253 - val_loss: 27060.4043 - val_mae: 164.4652 - val_mape: 98.7282
Epoch 4/50
6/6 [==============================] - 0s 26ms/step - loss: 23574.3848 - mae: 152.6142 - mape: 98.0018 - val_loss: 26674.8438 - val_mae: 163.2898 - val_mape: 98.0230
Epoch 5/50
6/6 [==============================] - 0s 17ms/step - loss: 23158.5723 - mae: 151.2379 - mape: 97.0930 - val_loss: 26160.4629 - val_mae: 161.7088 - val_mape: 97.0750
Epoch 6/50
6/6 [==============================] - 0s 18ms/step - loss: 22605.9004 - mae: 149.4148 - mape: 95.

In [9]:
for model in models:

  transformer = models[model][0]
  X_train = models[model][1]
  X_test = models[model][2]
  y_train = models[model][3]
  y_test = models[model][4]

  # Evaluate the model on the test set
  y_pred = transformer.predict(X_test)


  mse = mean_squared_error(y_test, y_pred)
  evals = transformer.evaluate(X_test, y_test)

  print(f"{model}")

  print(f'Mean Squared Error on Test Set: {mse}')
  print(f'Root Mean Squared Error on Test Set: {mse**(1/2)}')
  print(f'Mean Absolute Error on Test Set: {evals[1]}')
  print(f'Mean Absolute Percentage Error on Test Set: {evals[2]}\n\n')

2/2 [==============================] - 0s 14ms/step - loss: 74.2661 - mae: 7.6158 - mape: 4.1266
AAPL
Mean Squared Error on Test Set: 74.26611051102238
Root Mean Squared Error on Test Set: 8.617778745768678
Mean Absolute Error on Test Set: 7.615825653076172
Mean Absolute Percentage Error on Test Set: 4.126607894897461


3/3 [==============================] - 0s 5ms/step - loss: 648.6970 - mae: 21.7743 - mape: 16.9434
AMZN
Mean Squared Error on Test Set: 648.6970034859165
Root Mean Squared Error on Test Set: 25.469530884684872
Mean Absolute Error on Test Set: 21.774253845214844
Mean Absolute Percentage Error on Test Set: 16.94340705871582


3/3 [==============================] - 0s 5ms/step - loss: 537.4771 - mae: 16.4564 - mape: 5.3388
MSFT
Mean Squared Error on Test Set: 537.4772570860044
Root Mean Squared Error on Test Set: 23.18355574725336
Mean Absolute Error on Test Set: 16.456403732299805
Mean Absolute Percentage Error on Test Set: 5.338837146759033


